# XSLT partial code generator for transforming Infopad form xml
## The Conditions are listed below:

|      |  Condition  | Group | Question | Answer |
|:----:|:-----------:|:-----:|:--------:|:------:|
| n.a. | Condition 0 |   0   |     0    |    0   |
|      | Condition 1 |   0   |     0    |    1   |
|      | Condition 2 |   0   |     1    |    0   |
|      | Condition 3 |   0   |     1    |    1   |
|      | Condition 4 |   1   |     0    |    0   |
| n.a. | Condition 5 |   1   |     0    |    1   |
|      | Condition 6 |   1   |     1    |    0   |
|      | Condition 7 |   1   |     1    |    1   |

In [1]:
# This makes the plots more crisp
# To enable the autocomplete feature based upon clicking the TAB button
%config IPCompleter.greedy=True
%config InlineBackend.figure_format = 'retina'
%matplotlib inline 

In [2]:
import pandas as pd
import numpy as np
from decimal import *
import matplotlib.pyplot as plt

In [3]:
raw = pd.read_excel('infopad-ism-variable-mapping_20201102_v1_IM.xlsx')

In [4]:
df_raw = raw.copy(deep=True)
df_raw.head()   # size 922 x 8

,codebook_id,infopad_task_id,infopad_task_txt,infopad_form_txt,infopad_group_txt,infopad_question_txt,infopad_answer_txt,ism_variable,is_var
0,2.0,249961490.0,NaN,NaN,NaN,Klinikk,NaN,physio_id,1.0
1,3.0,249961490.0,NaN,NaN,NaN,Pasient nr,NaN,journal_id,1.0
2,4.0,249961490.0,NaN,NaN,NaN,Fødselsår,NaN,birth_year,1.0
3,5.0,249961490.0,SupportPRIM - Baseline Pasient,Bakgrunnsdata - supportPRIM,Bakgrunnsdata,Hvordan vil du beskrive arbeidet ditt (lønnet ...,NaN,work_type_1,1.0
4,9.0,249961490.0,SupportPRIM - Baseline Pasient,Bakgrunnsdata - supportPRIM,Bakgrunnsdata,Arbeidsevne,NaN,work_ability_1,1.0


# 
# 
## Write the formated xslt code to a file

In [5]:
def save_xslt_to_file(xslt_code, file_name='xslt_code'):
    with open(file_name+'.xslt', 'w') as f:
        print(xslt_code, file=f)

## 

In [6]:
df = df_raw.dropna( subset=['infopad_question_txt', 'infopad_answer_txt', 'ism_variable'], how='all')
df.head()  # size 506 x 7

,codebook_id,infopad_task_id,infopad_group_txt,infopad_question_txt,infopad_answer_txt,ism_variable,is_var
1,2.0,249961490.0,NaN,Klinikk,NaN,physio_id,1.0
2,3.0,249961490.0,NaN,Pasient nr,NaN,journal_id,1.0
3,4.0,249961490.0,NaN,Fødselsår,NaN,birth_year,1.0
4,5.0,249961490.0,NaN,Hvordan vil du beskrive arbeidet ditt (lønnet ...,NaN,work_type_1,1.0
5,6.0,249961490.0,NaN,NaN,NaN,work_ability_1,NaN


In [7]:
def debug_print(codebook_id='', infopad_task_id='', infopad_group_txt='', infopad_question_txt='', infopad_answere_txt='',  ism_variable='', is_var=''):
    print(
            codebook_id,          '   :  \t', 
            infopad_task_id,       ' \t:  \t', 
            infopad_group_txt,    ' \t:  \t', 
            infopad_question_txt, ' \t:  \t', 
            infopad_answere_txt,  ' \t:  \t', 
            ism_variable,         ' \t:  \t', 
            is_var
    ) 

In [8]:
def condition_df_appender(
    df,
    codebook_id, 
    infopad_task_id,
    infopad_group_txt, 
    infopad_question_txt, 
    infopad_answer_txt, 
    ism_variable,
    is_var):
    
    df = df.append(
                    {
                        'codebook_id'           : codebook_id,   
                        'infopad_task_id'       : str(int(infopad_task_id)),
                        'infopad_group_txt'     : infopad_group_txt,              
                        'infopad_question_txt'  : infopad_question_txt,              
                        'infopad_answer_txt'    : infopad_answer_txt,                          
                        'ism_variable'          : ism_variable,      
                        'is_var'                : int(is_var)  
                   }, 
                   ignore_index=True)
    return df

## Logic for only extracting the group combination for creating the ism_variable

In [9]:
def populate_condition_df(
    condition_df_list,
    codebook_id, 
    infopad_task_id,
    infopad_group_txt, 
    infopad_question_txt, 
    infopad_answer_txt, 
    ism_variable,
    is_var):
    
    if ( is_var == 1)\
    and (infopad_task_id > 0):
    
        if  ( isinstance( infopad_group_txt    , str)  ==  False ) :
            if  ( isinstance( infopad_question_txt , str)  ==  False ) :
                
                if  ( isinstance( infopad_answer_txt   , str)  ==  False  ) :
                    #print('\n Error........................ \n')
                    condition_df_list[0] = condition_df_appender( condition_df_list[0], codebook_id, infopad_task_id, infopad_group_txt,
                                                                 infopad_question_txt, infopad_answer_txt, ism_variable, is_var)
                    
                    
                elif ( isinstance( infopad_answer_txt   , str)  ==  True  ) :
                    #print('\nCondition 2: ')
                    #debug_print(codebook_id, '', '', '', ism_group_var, ism_question_var, ism_answer_var, ism_variable)
                    condition_df_list[1] = condition_df_appender( condition_df_list[1], codebook_id, infopad_task_id, infopad_group_txt,
                                                                 infopad_question_txt, infopad_answer_txt, ism_variable, is_var)
                    #print(condition_df_list[1])
                    
                    
            elif ( isinstance( infopad_question_txt , str)  ==  True ) :
                
                if  ( isinstance( infopad_answer_txt   , str)  ==  False  ) :
                    #print('\n Condition 3: ', ism_question_var, '\t: ',ism_answer_var)
                    #debug_print(codebook_id, '', '', '', ism_group_var, ism_question_var, ism_answer_var, ism_variable)
                    condition_df_list[2] = condition_df_appender( condition_df_list[2], codebook_id, infopad_task_id, infopad_group_txt,
                                                                 infopad_question_txt, infopad_answer_txt, ism_variable, is_var)
                    
                elif ( isinstance( infopad_answer_txt   , str)  ==  True  ) :
                    #print('\n Condition 4: ', ism_question_var, '\t: ',ism_answer_var)
                    #debug_print(codebook_id, '', '', '', ism_group_var, ism_question_var, ism_answer_var, ism_variable)
                    condition_df_list[3] = condition_df_appender( condition_df_list[3], codebook_id, infopad_task_id, infopad_group_txt,
                                                                 infopad_question_txt, infopad_answer_txt, ism_variable, is_var)
                
                
                
        elif ( isinstance( infopad_group_txt    , str)  ==  True ) :
            if  ( isinstance( infopad_question_txt , str)  ==  False ) :
                
                if  ( isinstance( infopad_answer_txt   , str)  ==  False  ) :
                    #print('\n Condition 5: ', ism_question_var, '\t: ',ism_answer_var)
                    #debug_print(codebook_id, '', '', '', ism_group_var, ism_question_var, ism_answer_var, ism_variable)
                    condition_df_list[4] = condition_df_appender( condition_df_list[4], codebook_id, infopad_task_id, infopad_group_txt,
                                                                 infopad_question_txt, infopad_answer_txt, ism_variable, is_var)
                    
                elif ( isinstance( infopad_answer_txt   , str)  ==  True  ) :
                    #gen_answer_xslt()
                    #print('\n Condition 6: ', ism_question_var, '\t: ',ism_answer_var)
                    #debug_print(codebook_id, '', '', '', ism_group_var, ism_question_var, ism_answer_var, ism_variable)
                    condition_df_list[5] = condition_df_appender( condition_df_list[5], codebook_id, infopad_task_id, infopad_group_txt,
                                                                 infopad_question_txt, infopad_answer_txt, ism_variable, is_var)
                    
                    
            elif ( isinstance( infopad_question_txt , str)  ==  True ) :
                
                if  ( isinstance( infopad_answer_txt   , str)  ==  False  ) :
                    #print('\n Condition 7: ', ism_question_var, '\t: ',ism_answer_var)
                    #debug_print(codebook_id, '', '', '', ism_group_var, ism_question_var, ism_answer_var, ism_variable)
                    condition_df_list[6] = condition_df_appender( condition_df_list[6], codebook_id, infopad_task_id, infopad_group_txt,
                                                                 infopad_question_txt, infopad_answer_txt, ism_variable, is_var)
                    
                if  ( isinstance( infopad_answer_txt   , str)  ==  True  ) :
                    #print('\n Condition 8: ', ism_question_var, '\t: ',ism_answer_var)
                    #debug_print(codebook_id, '', '', '', ism_group_var, ism_question_var, ism_answer_var, ism_variable)
                    condition_df_list[7] = condition_df_appender( condition_df_list[7], codebook_id, infopad_task_id, infopad_group_txt,
                                                                 infopad_question_txt, infopad_answer_txt, ism_variable, is_var)


In [10]:
pd.DataFrame(columns=df.columns.tolist())

,codebook_id,infopad_task_id,infopad_group_txt,infopad_question_txt,infopad_answer_txt,ism_variable,is_var


In [11]:
pd.DataFrame(columns=['codebook_id', 'infopad_task_id', 'infopad_group_txt', 'infopad_question_txt', 'infopad_answer_txt', 'ism_variable','is_var'])

,codebook_id,infopad_task_id,infopad_group_txt,infopad_question_txt,infopad_answer_txt,ism_variable,is_var


In [12]:
df_cond_0 = pd.DataFrame(columns=df.columns.tolist())
df_cond_1 = pd.DataFrame(columns=df.columns.tolist())
df_cond_2 = pd.DataFrame(columns=df.columns.tolist())
df_cond_3 = pd.DataFrame(columns=df.columns.tolist())
df_cond_4 = pd.DataFrame(columns=df.columns.tolist())
df_cond_5 = pd.DataFrame(columns=df.columns.tolist())
df_cond_6 = pd.DataFrame(columns=df.columns.tolist())
df_cond_7 = pd.DataFrame(columns=df.columns.tolist())


condition_df_list = [df_cond_0, df_cond_1, df_cond_2, df_cond_3, df_cond_4, df_cond_5, df_cond_6, df_cond_7]
zip_df = zip(df.iloc[:, 0], df.iloc[:, 1], df.iloc[:, 2], df.iloc[:, 3], df.iloc[:, 4], df.iloc[:, 5], df.iloc[:, 6])

for codebook_id, infopad_task_id, infopad_group_txt, infopad_question_txt, infopad_answer_txt, ism_variable, is_var in zip_df:
    #print(infopad_xml_text, ism_group_var, ism_question_var, ism_answer_var, ism_variable)
    
    cbr_empty_val = '_unknown_'
    
    infopad_empty = 'infopad_var_not_defined'
    ism_empty = 'ism_var_not_defined'
    
    if  ( isinstance( ism_variable   , str)  ==  False  ) :
        ism_variable = ism_empty
    
    populate_condition_df( condition_df_list, codebook_id, infopad_task_id, infopad_group_txt, infopad_question_txt, infopad_answer_txt, ism_variable, is_var)

In [13]:
for df_test in condition_df_list:
    print( df_test.shape)
    
# Output 
# (0, 7)
# (0, 7)
# (239, 7)
# (55, 7)
# (36, 7)
# (0, 7)
# (37, 7)
# (85, 7)

(0, 7)
(0, 7)
(238, 7)
(55, 7)
(36, 7)
(0, 7)
(37, 7)
(85, 7)


In [14]:
xslt_code_pre = "<?xml version=\"1.0\" encoding=\"UTF-8\"?>\n"+\
                "    <xsl:stylesheet version=\"1.0\" xmlns:xsl=\"http://www.w3.org/1999/XSL/Transform\">\n"+\
                "        <xsl:output encoding=\"UTF-8\" indent=\"yes\" method=\"xml\" version=\"1.0\"/>\n\n"+\
                "        <xsl:template match=\"/\">\n"+\
                "            <patient>\n\n"+\
                "                <projectId><xsl:value-of select=\"forms/@projectnumber\"/></projectId>\n"+\
                "                <taskId>249961490</taskId>\n"+\
                "                <taskName>SupportPRIM - Baseline Pasient</taskName>\n"+\
                "                <clinicName><xsl:value-of select=\"forms/form/patient/@medicalOffice\"/></clinicName>\n"+\
                "                <journalId><xsl:value-of select=\"forms/form/patient/@journalId\"/></journalId>\n"+\
                "                <birthYear><xsl:value-of select=\"forms/form/patient/@birthYear\"/></birthYear>\n\n"+\
                "                <!--Get the star date and time-->\n"+\
                "                <xsl:for-each select=\"//form\">\n"+\
                "                    <xsl:if test=\"@Name = 'Bakgrunnsdata - supportPRIM'\">\n"+\
                "                        <startDate>"+\
                "                            <xsl:value-of select=\"substring-before(@start, ' ')\"/>\n"+\
                "                        </startDate>\n"+\
                "                        <startTime>\n"+\
                "                            <xsl:value-of select=\"substring-after(@start, ' ')\"/>\n"+\
                "                        </startTime>\n"+\
                "                    </xsl:if>\n"+\
                "                </xsl:for-each>\n\n"+\
                "                <xsl:for-each select=\"//form/groups/group\">\n"
    
print(xslt_code_pre)

<?xml version="1.0" encoding="UTF-8"?>
    <xsl:stylesheet version="1.0" xmlns:xsl="http://www.w3.org/1999/XSL/Transform">
        <xsl:output encoding="UTF-8" indent="yes" method="xml" version="1.0"/>

        <xsl:template match="/">
            <patient>

                <projectId><xsl:value-of select="forms/@projectnumber"/></projectId>
                <taskId>249961490</taskId>
                <taskName>SupportPRIM - Baseline Pasient</taskName>
                <clinicName><xsl:value-of select="forms/form/patient/@medicalOffice"/></clinicName>
                <journalId><xsl:value-of select="forms/form/patient/@journalId"/></journalId>
                <birthYear><xsl:value-of select="forms/form/patient/@birthYear"/></birthYear>

                <!--Get the star date and time-->
                <xsl:for-each select="//form">
                    <xsl:if test="@Name = 'Bakgrunnsdata - supportPRIM'">
                        <startDate>                            <xsl:value-of select="

In [15]:
xslt_code_post = "            </xsl:for-each>\n"+\
                 "        </patient>\n"+\
                 "    </xsl:template>\n"+\
                 "</xsl:stylesheet>"
print(xslt_code_post)

            </xsl:for-each>
        </patient>
    </xsl:template>
</xsl:stylesheet>


# 
# 
## Generate XSLT for the combination of None from Infopad xml
### (Condition 0)

In [16]:
condition_df_list[0]

,codebook_id,infopad_task_id,infopad_group_txt,infopad_question_txt,infopad_answer_txt,ism_variable,is_var


In [17]:
def gen_condition_0_xslt(df, empty_val='_unknown_'):
    xslt = "   <!-- Code for condition : 0  START ....................... -->\n\n"

    for infopad_answer_txt, ism_variable in zip(df.infopad_answer_txt, df.ism_variable):
        #print ( infopad_answer_txt, ism_answer_var)

        xslt += "          <xsl:if test=\"answer[@selected='true'] and answer='"+infopad_answer_txt+"'\">\n"+\
                "             <"+ism_variable+"><xsl:value-of select=\"answer[@selected='true']\"/></"+ism_variable+">\n"+\
                "          </xsl:if>\n"
        
    xslt += "\n   <!-- Code for condition : 0  ......................... END -->\n\n\n"
    return xslt

print(gen_condition_0_xslt(condition_df_list[0]))

   <!-- Code for condition : 0  START ....................... -->


   <!-- Code for condition : 0  ......................... END -->





In [18]:
xslt_code_c0 = gen_condition_0_xslt(condition_df_list[0])

# 
# 
## Generate XSLT for the combination of Answer from Infopad xml
### (Condition 1)

In [19]:
condition_df_list[1][0:2]

,codebook_id,infopad_task_id,infopad_group_txt,infopad_question_txt,infopad_answer_txt,ism_variable,is_var


In [20]:
def gen_answer_xslt(df, empty_val='_unknown_'):
    xslt = "   <!-- Code for condition : 1  START ....................... -->\n\n"
    for infopad_answer_txt, ism_variable in zip(df.infopad_answer_txt, df.ism_variable):
        #print ( infopad_answer_txt, ism_variable)

        xslt +=  "<xsl:if test=\"answer[@selected='true'] and answer='"+infopad_answer_txt+"'\">\n"+\
                 "    <"+ism_variable+"><xsl:value-of select=\"answer[@selected='true']\"/></"+ism_variable+">\n"+\
                 "</xsl:if>\n"

    xslt += "\n   <!-- Code for condition : 1  ......................... END -->\n\n\n"
    return xslt

print(gen_answer_xslt(condition_df_list[1][0:1]))

   <!-- Code for condition : 1  START ....................... -->


   <!-- Code for condition : 1  ......................... END -->





In [21]:
xslt_code_c1 = gen_answer_xslt(condition_df_list[1])

# 
# 
## Generate XSLT for the combination of Question from Infopad xml
### (Condition 2)

In [22]:
condition_df_list[2][0:5]

,codebook_id,infopad_task_id,infopad_group_txt,infopad_question_txt,infopad_answer_txt,ism_variable,is_var
0,2.0,249961490,NaN,Klinikk,NaN,physio_id,1
1,3.0,249961490,NaN,Pasient nr,NaN,journal_id,1
2,4.0,249961490,NaN,Fødselsår,NaN,birth_year,1
3,5.0,249961490,NaN,Hvordan vil du beskrive arbeidet ditt (lønnet ...,NaN,work_type_1,1
4,9.0,249961490,NaN,Arbeidsevne,NaN,work_ability_1,1


In [23]:
def gen_question_xslt(df, empty_key='_unknown_', empty_val='_unknown_'):
    xslt = "   <!-- Code for condition : 2  START ....................... -->\n\n"
    
    xslt +=     "    <xsl:for-each select=\"question\">\n"
    for infopad_task_id, infopad_question_txt, ism_variable in zip(df.infopad_task_id, df.infopad_question_txt, df.ism_variable):
        
        if isinstance(infopad_question_txt, str) == False:
            infopad_question_txt = empty_val
        if isinstance(ism_variable, str) == False:
            ism_variable = empty_key
            
        xslt += "     <xsl:if test=\"../../../@taskid = '"+infopad_task_id+"'\">\n"+\
                "        <xsl:if test=\"@name='"+infopad_question_txt+"'\">\n"+\
                "            <"+ism_variable+">\n"+\
                "                <xsl:choose>\n"+\
                "                    <xsl:when test=\"answer/@selected='true'\">\n"+\
                "                        <xsl:value-of select=\"answer[@selected='true']\"/>\n"+\
                "                    </xsl:when>\n"+\
                "                    <xsl:when test=\"answer[not(@selected)]\">\n"+\
                "                        <xsl:value-of select=\"answer\"/>\n"+\
                "                    </xsl:when>\n"\
                "                    <xsl:otherwise>"+empty_val+"</xsl:otherwise>\n"+\
                "                </xsl:choose>\n"+\
                "            </"+ism_variable+">\n"+\
                "        </xsl:if>\n"+\
                "     </xsl:if>\n"
        
    xslt +=     "    </xsl:for-each>\n"
    
    xslt += "\n   <!-- Code for condition : 2  ......................... END -->\n\n\n"
    return xslt

print(gen_question_xslt(condition_df_list[2][0:1]))

   <!-- Code for condition : 2  START ....................... -->

    <xsl:for-each select="question">
     <xsl:if test="../../../@taskid = '249961490'">
        <xsl:if test="@name='Klinikk'">
            <physio_id>
                <xsl:choose>
                    <xsl:when test="answer/@selected='true'">
                        <xsl:value-of select="answer[@selected='true']"/>
                    </xsl:when>
                    <xsl:when test="answer[not(@selected)]">
                        <xsl:value-of select="answer"/>
                    </xsl:when>
                    <xsl:otherwise>_unknown_</xsl:otherwise>
                </xsl:choose>
            </physio_id>
        </xsl:if>
     </xsl:if>
    </xsl:for-each>

   <!-- Code for condition : 2  ......................... END -->





In [24]:
xslt_code_c2 = gen_question_xslt(condition_df_list[2])

# 
# 
## Generate XSLT for the combination of Question & Answer from Infopad xml
### (Condition 3)

In [25]:
condition_df_list[3].head()

,codebook_id,infopad_task_id,infopad_group_txt,infopad_question_txt,infopad_answer_txt,ism_variable,is_var
0,16.0,249961490,NaN,Andre nåværende sykdommer eller plager:,Revmatisk sykdom,como_ra_1,1
1,17.0,249961490,NaN,Andre nåværende sykdommer eller plager:,Hjertesykdom,como_heart_1,1
2,18.0,249961490,NaN,Andre nåværende sykdommer eller plager:,Astma/ lungesykdom,como_asthma_1,1
3,19.0,249961490,NaN,Andre nåværende sykdommer eller plager:,Nevrologisk sykdom (f.eks tidligere hjerneslag...,como_neuro_1,1
4,20.0,249961490,NaN,Andre nåværende sykdommer eller plager:,Kreft/ tidligere kreft,como_cancer_1,1


In [26]:
def gen_question_answer_xslt(df, empty_key='_unknown_', empty_val='_unknown_'):
    xslt = "   <!-- Code for condition : 3  START ....................... -->\n\n"
    
    xslt +=     "    <xsl:for-each select=\"question\">\n"
    for infopad_task_id, infopad_question_txt, infopad_answer_txt, ism_variable in zip( df.infopad_task_id, df.infopad_question_txt,	df.infopad_answer_txt, df.ism_variable):
        
        if isinstance(infopad_question_txt, str) == False:
            infopad_question_txt = empty_val
        if isinstance(infopad_answer_txt, str) == False:
            infopad_answer_txt = empty_val
        if isinstance(ism_variable, str) == False:
            ism_variable = empty_key
            
        xslt += "     <xsl:if test=\"../../../@taskid = '"+infopad_task_id+"'\">\n"+\
                "        <xsl:if test=\"@name = '"+infopad_question_txt+"'\">\n"+\
                "            <xsl:for-each select=\"answer\">\n"+\
                "                <xsl:if test=\".='"+infopad_answer_txt+"'\">\n"+\
                "                     <"+ism_variable+"><xsl:value-of select=\"./@selected\"/></"+ism_variable+">\n"+\
                "                 </xsl:if>\n"+\
                "            </xsl:for-each>\n"+\
                "        </xsl:if>\n"+\
                "     </xsl:if>\n"
                
    xslt +=     "    </xsl:for-each>\n"
    
    xslt += "\n   <!-- Code for condition : 3  ......................... END -->\n\n\n"
    return xslt
        
print(gen_question_answer_xslt(condition_df_list[3][0:1]))

   <!-- Code for condition : 3  START ....................... -->

    <xsl:for-each select="question">
     <xsl:if test="../../../@taskid = '249961490'">
        <xsl:if test="@name = 'Andre nåværende sykdommer eller plager:'">
            <xsl:for-each select="answer">
                <xsl:if test=".='Revmatisk sykdom'">
                     <como_ra_1><xsl:value-of select="./@selected"/></como_ra_1>
                 </xsl:if>
            </xsl:for-each>
        </xsl:if>
     </xsl:if>
    </xsl:for-each>

   <!-- Code for condition : 3  ......................... END -->





In [27]:
xslt_code_c3 = gen_question_answer_xslt(condition_df_list[3])

# 
# 
## Generate XSLT for the combination of Group from Infopad xml
### (Condition 4)

In [28]:
def gen_group_xslt(df, empty_key='_unknown_', empty_val='_unknown_'):
    xslt = "   <!-- Code for condition : 4  START ....................... -->\n\n"
    
    for infopad_task_id, infopad_group_txt, ism_variable in zip( df.infopad_task_id, df.infopad_group_txt, df.ism_variable):
        if isinstance(infopad_group_txt, str) == False:
            infopad_group_txt = empty_val
        if isinstance(ism_variable, str) == False:
            ism_variable = empty_key

        xslt += "     <xsl:if test=\"../../@taskid = '"+infopad_task_id+"'\">\n"+\
                "     <xsl:if test=\"@name='"+infopad_group_txt+"'\">\n"+\
                "        <xsl:for-each select=\"question\">\n"+\
                "               <xsl:choose>\n"+\
                "                   <xsl:when test=\"answer/@selected='true'\">\n"+\
                "                       <"+ism_variable+"><xsl:value-of select=\"answer\"/></"+ism_variable+">\n"+\
                "                   </xsl:when>\n"+\
                "                   <xsl:when test=\"answer[not(@selected)]\">\n"+\
                "                       <"+ism_variable+"><xsl:value-of select=\"answer\"/></"+ism_variable+">\n"+\
                "                   </xsl:when>\n"\
                "               </xsl:choose>\n"+\
                "        </xsl:for-each>\n"+\
                "       </xsl:if>\n"+\
                "     </xsl:if>\n"
        
    xslt += "\n   <!-- Code for condition : 4  ......................... END -->\n\n\n"
    return xslt
        
print(gen_group_xslt(condition_df_list[4][0:1]))

   <!-- Code for condition : 4  START ....................... -->

     <xsl:if test="../../@taskid = '249966786'">
     <xsl:if test="@name='1. Redusert aktivitet, hvile, avlastning'">
        <xsl:for-each select="question">
               <xsl:choose>
                   <xsl:when test="answer/@selected='true'">
                       <treat_advice_rest_3><xsl:value-of select="answer"/></treat_advice_rest_3>
                   </xsl:when>
                   <xsl:when test="answer[not(@selected)]">
                       <treat_advice_rest_3><xsl:value-of select="answer"/></treat_advice_rest_3>
                   </xsl:when>
               </xsl:choose>
        </xsl:for-each>
       </xsl:if>
     </xsl:if>

   <!-- Code for condition : 4  ......................... END -->





In [29]:
xslt_code_c4 = gen_group_xslt(condition_df_list[4])

# 
# 
## Generate XSLT for the combination of Group and Question from Infopad xml
### (Condition 6)

In [30]:
xslt_code_c5 = ""

In [31]:
condition_df_list[6].head()

,codebook_id,infopad_task_id,infopad_group_txt,infopad_question_txt,infopad_answer_txt,ism_variable,is_var
0,342.0,249961492,Nakkeundersøkelse,Reduserte og/eller smertefulle aktive bevegels...,NaN,neck_move_1,1
1,355.0,249961492,Nakkeundersøkelse,Spurling test positiv,NaN,neck_spurling_1,1
2,363.0,249961492,Nakkeundersøkelse,Ev. kommentar nevrologiske funn,NaN,neck_neuro_other_1,1
3,364.0,249961492,Nakkeundersøkelse,Konklusjon av nakkevurdering,NaN,neck_conclusion_1,1
4,365.0,249961492,Nakkeundersøkelse,Ev. kommentarer:,NaN,neck_comment_1,1


In [32]:
def gen_group_question_xslt(df, empty_key='_unknown_', empty_val='_unknown_'):
    xslt = "   <!-- Code for condition : 6  START ....................... -->\n\n"
    
    for infopad_task_id, infopad_group_txt, infopad_question_txt, ism_variable in zip(df.infopad_task_id, df.infopad_group_txt, df.infopad_question_txt, df.ism_variable):
        
        if isinstance(infopad_group_txt, str) == False:
            infopad_group_txt = empty_val
        if isinstance(infopad_question_txt, str) == False:
            infopad_question_txt = empty_val
        if isinstance(ism_variable, str) == False:
            ism_variable = empty_key
            
        xslt += "     <xsl:if test=\"../../@taskid = '"+infopad_task_id+"'\">\n"+\
                "      <xsl:if test=\"@name='"+infopad_group_txt+"'\">\n"+\
                "          <xsl:for-each select=\"question\">\n"+\
                "              <xsl:if test=\"@name = '"+infopad_question_txt+"'\">\n"+\
                "                  <"+ism_variable+">\n"+\
                "                      <xsl:choose>\n"+\
                "                         <xsl:when test=\"answer/@selected = 'true'\">\n"+\
                "                             <xsl:value-of select=\"answer[@selected = 'true']\"/>\n"+\
                "                         </xsl:when>\n"+\
                "                         <xsl:when test=\"answer[not(@selected)]\">\n"+\
                "                             <xsl:value-of select=\"answer\"/>\n"+\
                "                         </xsl:when>\n"\
                "                         <xsl:otherwise>"+empty_val+"</xsl:otherwise>\n"+\
                "                      </xsl:choose>\n"+\
                "                  </"+ism_variable+">\n"+\
                "              </xsl:if>\n"+\
                "         </xsl:for-each>\n"+\
                "        </xsl:if>\n"+\
                "     </xsl:if>\n"
        
    xslt += "\n   <!-- Code for condition : 6  ......................... END -->\n\n\n"
    return xslt
                          
print(gen_group_question_xslt(condition_df_list[6][0:1]))

   <!-- Code for condition : 6  START ....................... -->

     <xsl:if test="../../@taskid = '249961492'">
      <xsl:if test="@name='Nakkeundersøkelse'">
          <xsl:for-each select="question">
              <xsl:if test="@name = 'Reduserte og/eller smertefulle aktive bevegelsesutslag i vektbærende stilling'">
                  <neck_move_1>
                      <xsl:choose>
                         <xsl:when test="answer/@selected = 'true'">
                             <xsl:value-of select="answer[@selected = 'true']"/>
                         </xsl:when>
                         <xsl:when test="answer[not(@selected)]">
                             <xsl:value-of select="answer"/>
                         </xsl:when>
                         <xsl:otherwise>_unknown_</xsl:otherwise>
                      </xsl:choose>
                  </neck_move_1>
              </xsl:if>
         </xsl:for-each>
        </xsl:if>
     </xsl:if>

   <!-- Code for condition : 6  .......

In [33]:
xslt_code_c6 = gen_group_question_xslt(condition_df_list[6])

# 
# 
## Generate XSLT for the combination of Group, Question, and Answer from Infopad xml
### (Condition 7)

In [34]:
condition_df_list[7].head()

,codebook_id,infopad_task_id,infopad_group_txt,infopad_question_txt,infopad_answer_txt,ism_variable,is_var
0,344.0,249961492,Nakkeundersøkelse,Reduserte bevegelsesutslag i,Fleksjon,neck_move_rom_flex_1,1
1,345.0,249961492,Nakkeundersøkelse,Reduserte bevegelsesutslag i,Ekstensjon,neck_move_rom_ext_1,1
2,346.0,249961492,Nakkeundersøkelse,Reduserte bevegelsesutslag i,Rotasjon,neck_move_rom_rot_1,1
3,347.0,249961492,Nakkeundersøkelse,Reduserte bevegelsesutslag i,Lateralfleksjon,neck_move_rom_lat_1,1
4,348.0,249961492,Nakkeundersøkelse,Reduserte bevegelsesutslag i,Ikke reduserte bevegelsesutslag,neck_move_rom_not_1,1


In [35]:
def gen_group_question_answer_xslt(df, empty_val='_unknown_'):
    xslt = "   <!-- Code for condition : 7  START ....................... -->\n\n"

    for infopad_task_id, infopad_group_txt, infopad_question_txt, infopad_answer_txt, ism_variable in zip( 
        df.infopad_task_id, df.infopad_group_txt, df.infopad_question_txt,	df.infopad_answer_txt, df.ism_variable):
        
        if isinstance(infopad_group_txt, str) == False:
            infopad_group_txt = empty_val
        if isinstance(infopad_question_txt, str) == False:
            infopad_question_txt = empty_val
        if isinstance(infopad_answer_txt, str) == False:
            infopad_answer_txt = empty_val
        if isinstance(ism_variable, str) == False:
            ism_variable = empty_key
    
        xslt += "     <xsl:if test=\"../../@taskid = '"+infopad_task_id+"'\">\n"+\
                "      <xsl:if test=\"@name = '"+infopad_group_txt+"'\">\n"+\
                "          <xsl:for-each select=\"question\">\n"+\
                "              <xsl:if test=\"@name = '"+infopad_question_txt+"'\">\n"+\
                "                  <xsl:for-each select=\"answer\">\n"+\
                "                      <xsl:if test=\".='"+infopad_answer_txt+"'\">\n"+\
                "                           <"+ism_variable+"><xsl:value-of select=\"./@selected\"/></"+ism_variable+">\n"+\
                "                      </xsl:if>\n"+\
                "                  </xsl:for-each>\n"+\
                "              </xsl:if>\n"+\
                "          </xsl:for-each>\n"+\
                "        </xsl:if>\n"+\
                "     </xsl:if>\n"
        
    xslt += "\n   <!-- Code for condition : 7  ......................... END -->\n\n\n"
    return xslt

print(gen_group_question_answer_xslt(condition_df_list[7][0:1]))

   <!-- Code for condition : 7  START ....................... -->

     <xsl:if test="../../@taskid = '249961492'">
      <xsl:if test="@name = 'Nakkeundersøkelse'">
          <xsl:for-each select="question">
              <xsl:if test="@name = 'Reduserte bevegelsesutslag i'">
                  <xsl:for-each select="answer">
                      <xsl:if test=".='Fleksjon'">
                           <neck_move_rom_flex_1><xsl:value-of select="./@selected"/></neck_move_rom_flex_1>
                      </xsl:if>
                  </xsl:for-each>
              </xsl:if>
          </xsl:for-each>
        </xsl:if>
     </xsl:if>

   <!-- Code for condition : 7  ......................... END -->





In [36]:
xslt_code_c7 = gen_group_question_answer_xslt(condition_df_list[7])

In [38]:
final_xslt = xslt_code_pre
final_xslt += xslt_code_c0 + xslt_code_c1 + xslt_code_c2 + xslt_code_c3 + xslt_code_c4 + xslt_code_c5 + xslt_code_c6 + xslt_code_c7 
final_xslt += xslt_code_post

In [39]:
save_xslt_to_file( xslt_code=final_xslt, file_name='xslt_code_for_infopad_200701')